In [1]:
import re  # For preprocessing
import pandas as pd  # For data handling
from time import time  # To time our operations
from collections import defaultdict  # For word frequency
import numpy as np
import pandas as pd

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import spacy
from spacy.lemmatizer import Lemmatizer
from spacy.lang.en.stop_words import STOP_WORDS
import en_core_web_lg

from tqdm import tqdm_notebook as tqdm
from pprint import pprint
import spacy  # For preprocessing
# from pyLDAvis import gensim
from time import time 
import numpy as np
import pandas as pd
from IPython.display import display
from tqdm import tqdm
from collections import Counter
import ast

import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import seaborn as sb

from sklearn.feature_extraction.text import CountVectorizer
from textblob import TextBlob
import scipy.stats as stats
from scipy.sparse import csr_matrix
import spacy

from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.manifold import TSNE

import pyLDAvis

from bokeh.plotting import figure, output_file, show
from bokeh.models import Label
from bokeh.io import output_notebook
output_notebook()



import logging  # Setting up the loggings to monitor gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

nlp = spacy.load('en', disable=['ner', 'parser']) # disabling Named Entity Recognition for speed
%matplotlib inline

Loading BokehJS ...

C:\Users\shile\Anaconda3\envs\nlp_course\lib\site-packages\msgpack_numpy.py:184: DeprecationWarning: encoding is deprecated, Use raw=False instead.
  return _unpackb(packed, **kwargs)
C:\Users\shile\Anaconda3\envs\nlp_course\lib\site-packages\msgpack_numpy.py:184: DeprecationWarning: encoding is deprecated, Use raw=False instead.
  return _unpackb(packed, **kwargs)


In [2]:
import numpy as np
b=np.load('ny.eater-content-new.npy')
content=b.tolist()

In [3]:
def cleaning(doc):
    # Lemmatizes and removes stopwords
    # doc needs to be a spacy Doc object
    txt = [token.lemma_ for token in doc if not token.is_stop]
    # Word2Vec uses context words to learn the vector representation of a target word,
    # if a sentence is only one or two words long,
    # the benefit for the training is very small
    if len(txt) > 2:
        return ' '.join(txt)

new_content = []
for article in content:
    if article != []:
        text = ""
        for sent in article:
            text = text + sent
        new_content.append(text)



In [4]:
brief_cleaning = (re.sub("[^A-Za-z']+", ' ', re.sub("'s", '', re.sub("’s", '',str(article)))).lower() for article in new_content)

t = time()

txt = [cleaning(doc) for doc in nlp.pipe(brief_cleaning, batch_size=5000, n_threads=-1)]

print('Time to clean up everything: {} mins'.format(round((time() - t) / 60, 2)))

df_clean = pd.DataFrame({'clean': txt})
df_clean = df_clean.dropna().drop_duplicates()
df_clean.shape

Time to clean up everything: 0.41 mins


(395, 1)

In [5]:
from gensim.models.phrases import Phrases, Phraser
sent = [row.split() for row in df_clean['clean']]
bigram = Phrases(sent, min_count=30, progress_per=10000)
sentences = bigram[sent]

INFO - 17:44:34: collecting all words and their counts
INFO - 17:44:34: PROGRESS: at sentence #0, processed 0 words and 0 word types
INFO - 17:44:34: collected 239572 word types from a corpus of 283344 words (unigram + bigrams) and 395 sentences
INFO - 17:44:34: using 239572 counts as vocab in Phrases<0 vocab, min_count=30, threshold=10.0, max_vocab_size=40000000>


In [6]:
doc_list=sentences
# Creates, which is a mapping of word IDs to words.
words = corpora.Dictionary(doc_list)

# Turns each document into a bag of words.
corpus = [words.doc2bow(doc) for doc in doc_list]

INFO - 17:44:34: adding document #0 to Dictionary(0 unique tokens: [])
INFO - 17:44:36: built Dictionary(22942 unique tokens: ['abalone', 'absolutely', 'absurd', 'accessible', 'acidity']...) from 395 documents (total 275313 corpus positions)


In [7]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=words,
                                           num_topics=20, 
                                           random_state=2,
                                           update_every=1,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

INFO - 17:44:37: using autotuned alpha, starting with [0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05]
INFO - 17:44:37: using symmetric eta at 0.05
INFO - 17:44:37: using serial LDA version on this node
INFO - 17:44:37: running online (multi-pass) LDA training, 20 topics, 10 passes over the supplied corpus of 395 documents, updating model once every 395 documents, evaluating perplexity every 395 documents, iterating 50x with a convergence threshold of 0.001000
INFO - 17:44:41: -13.429 per-word bound, 11032.3 perplexity estimate based on a held-out corpus of 395 documents with 275313 words
INFO - 17:44:41: PROGRESS: pass 0, at document #395/395
INFO - 17:44:43: optimized alpha [0.09346892, 0.09309897, 0.09423925, 0.094810836, 0.0932745, 0.09369795, 0.08834974, 0.090643354, 0.09448183, 0.09388143, 0.08897023, 0.09605072, 0.0954025, 0.09496476, 0.09458937, 0.09035362, 0.09659594, 0.09462388, 0.094126776, 0.094926566]


INFO - 17:45:11: topic #15 (0.032): 0.007*"tajine" + 0.004*"couscous" + 0.003*"omar" + 0.002*"chicken" + 0.002*"algerian" + 0.002*"bastilla" + 0.002*"north" + 0.002*"french" + 0.002*"restaurant" + 0.002*"african"
INFO - 17:45:11: topic #6 (0.032): 0.008*"curry" + 0.007*"sri" + 0.006*"lankan" + 0.004*"hopper" + 0.004*"san" + 0.004*"rasa" + 0.004*"chicken" + 0.003*"fish" + 0.003*"roti" + 0.003*"joe"
INFO - 17:45:11: topic #12 (0.051): 0.007*"restaurant" + 0.006*"like" + 0.005*"good" + 0.004*"dish" + 0.004*"food" + 0.003*"sauce" + 0.003*"meat" + 0.003*"sweet" + 0.003*"come" + 0.003*"chef"
INFO - 17:45:11: topic #11 (0.076): 0.010*"restaurant" + 0.006*"like" + 0.006*"dish" + 0.004*"menu" + 0.004*"food" + 0.004*"include" + 0.004*"chicken" + 0.004*"good" + 0.003*"open" + 0.003*"fry"
INFO - 17:45:11: topic #16 (0.178): 0.008*"restaurant" + 0.006*"like" + 0.005*"dish" + 0.004*"chef" + 0.004*"good" + 0.004*"menu" + 0.003*"food" + 0.003*"come" + 0.003*"bar" + 0.003*"flavor"
INFO - 17:45:11: topi

In [8]:
pprint(lda_model.print_topics(num_words=20))

INFO - 17:45:31: topic #0 (0.026): 0.007*"restaurant" + 0.005*"not" + 0.005*"good" + 0.005*"like" + 0.004*"spot" + 0.004*"cardoz" + 0.004*"carbone" + 0.004*"white" + 0.004*"beef" + 0.003*"bread" + 0.003*"rice" + 0.003*"meat" + 0.003*"flavor" + 0.003*"food" + 0.003*"dish" + 0.003*"sauce" + 0.003*"steak" + 0.003*"new_york" + 0.003*"steakhouse" + 0.003*"chicken"
INFO - 17:45:31: topic #1 (0.027): 0.010*"like" + 0.009*"drink" + 0.008*"chicken" + 0.008*"sample_dish" + 0.007*"sauce" + 0.006*"salad" + 0.006*"dish" + 0.006*"sandwich" + 0.006*"cheese" + 0.005*"beer" + 0.005*"pizza" + 0.005*"fry" + 0.005*"noodle" + 0.005*"entree" + 0.005*"shrimp" + 0.005*"cream" + 0.005*"selection" + 0.004*"menu" + 0.004*"con" + 0.004*"garlic"
INFO - 17:45:31: topic #2 (0.029): 0.007*"pizza" + 0.006*"like" + 0.005*"pie" + 0.005*"restaurant" + 0.005*"wine" + 0.005*"menu" + 0.004*"open" + 0.004*"good" + 0.004*"chef" + 0.004*"expect" + 0.003*"come" + 0.003*"salad" + 0.003*"cheese" + 0.003*"red" + 0.003*"bar" + 0.00

[(0,
  '0.007*"restaurant" + 0.005*"not" + 0.005*"good" + 0.005*"like" + '
  '0.004*"spot" + 0.004*"cardoz" + 0.004*"carbone" + 0.004*"white" + '
  '0.004*"beef" + 0.003*"bread" + 0.003*"rice" + 0.003*"meat" + 0.003*"flavor" '
  '+ 0.003*"food" + 0.003*"dish" + 0.003*"sauce" + 0.003*"steak" + '
  '0.003*"new_york" + 0.003*"steakhouse" + 0.003*"chicken"'),
 (1,
  '0.010*"like" + 0.009*"drink" + 0.008*"chicken" + 0.008*"sample_dish" + '
  '0.007*"sauce" + 0.006*"salad" + 0.006*"dish" + 0.006*"sandwich" + '
  '0.006*"cheese" + 0.005*"beer" + 0.005*"pizza" + 0.005*"fry" + '
  '0.005*"noodle" + 0.005*"entree" + 0.005*"shrimp" + 0.005*"cream" + '
  '0.005*"selection" + 0.004*"menu" + 0.004*"con" + 0.004*"garlic"'),
 (2,
  '0.007*"pizza" + 0.006*"like" + 0.005*"pie" + 0.005*"restaurant" + '
  '0.005*"wine" + 0.005*"menu" + 0.004*"open" + 0.004*"good" + 0.004*"chef" + '
  '0.004*"expect" + 0.003*"come" + 0.003*"salad" + 0.003*"cheese" + '
  '0.003*"red" + 0.003*"bar" + 0.003*"player" + 0.003*"

In [9]:
lda_df = lda_model.get_document_topics(corpus,minimum_probability=0)
lda_df = pd.DataFrame(list(lda_df))
num_topics = lda_model.num_topics
lda_df.columns = ['Topic'+str(i) for i in range(num_topics)]
for i in range(len(lda_df.columns)):
    lda_df.iloc[:,i]=lda_df.iloc[:,i].apply(lambda x: x[1])
lda_df['Automated_topic_id'] =lda_df.apply(lambda x: np.argmax(x),axis=1)
lda_df.head()

C:\Users\shile\Anaconda3\envs\nlp_course\lib\site-packages\numpy\core\fromnumeric.py:56: FutureWarning: 'argmax' is deprecated, use 'idxmax' instead. The behavior of 'argmax'
will be corrected to return the positional maximum in the future.
Use 'series.values.argmax' to get the position of the maximum now.
  return getattr(obj, method)(*args, **kwds)


,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,Topic6,Topic7,Topic8,Topic9,...,Topic11,Topic12,Topic13,Topic14,Topic15,Topic16,Topic17,Topic18,Topic19,Automated_topic_id
0,0.000022,0.000023,0.000025,0.000025,0.000022,0.000024,0.000021,0.000021,0.000025,0.000023,...,0.092917,0.046725,0.000027,0.000025,0.000021,0.312709,0.000022,0.005823,0.541478,Topic19
1,0.000028,0.000029,0.000032,0.000031,0.000028,0.000030,0.000026,0.000027,0.000032,0.000029,...,0.000075,0.000043,0.000034,0.000032,0.000026,0.999370,0.000027,0.000033,0.000042,Topic16
2,0.000030,0.000031,0.000034,0.000033,0.000030,0.000032,0.000028,0.000028,0.000034,0.000031,...,0.000081,0.000046,0.000036,0.000034,0.000028,0.630613,0.000029,0.000035,0.368757,Topic16
3,0.000030,0.000031,0.000034,0.184576,0.000030,0.000032,0.000028,0.000029,0.000034,0.000032,...,0.000081,0.000047,0.000037,0.000034,0.000028,0.601083,0.000029,0.000035,0.213741,Topic16
4,0.000027,0.000027,0.000030,0.000029,0.000027,0.000028,0.000025,0.000025,0.000030,0.000028,...,0.000072,0.000041,0.119272,0.000030,0.000025,0.880161,0.000026,0.000031,0.000040,Topic16


In [10]:
# Compute Perplexity, a measure of how good the model is. lower the better.
print('\nPerplexity: ', lda_model.log_perplexity(corpus))
# Compute Coherence Score for lda model
coherence_model_lda = CoherenceModel(model=lda_model, texts=doc_list, dictionary=words, coherence='c_v')
coherence_lda_c_v = coherence_model_lda.get_coherence()
print('\nCoherence Score using c_v: ', coherence_lda_c_v)
# Compute Coherence Score for lda model
coherence_model_lda = CoherenceModel(model=lda_model, texts=doc_list, dictionary=words, coherence='u_mass')
coherence_lda_u_mass = coherence_model_lda.get_coherence()
print('\nCoherence Score using u_mass: ', coherence_lda_u_mass)

INFO - 17:45:36: -8.840 per-word bound, 458.3 perplexity estimate based on a held-out corpus of 395 documents with 275313 words
INFO - 17:45:36: using ParallelWordOccurrenceAccumulator(processes=11, batch_size=64) to estimate probabilities from sliding windows



Perplexity:  -8.840122531191453


INFO - 17:45:48: 1 batches submitted to accumulate stats from 64 documents (41855 virtual)
INFO - 17:45:48: 2 batches submitted to accumulate stats from 128 documents (77143 virtual)
INFO - 17:45:48: 3 batches submitted to accumulate stats from 192 documents (118880 virtual)
INFO - 17:45:48: 4 batches submitted to accumulate stats from 256 documents (157333 virtual)
INFO - 17:45:49: 5 batches submitted to accumulate stats from 320 documents (194634 virtual)
INFO - 17:45:49: 6 batches submitted to accumulate stats from 384 documents (225682 virtual)
INFO - 17:45:49: 7 batches submitted to accumulate stats from 448 documents (232258 virtual)
INFO - 17:45:51: 11 accumulators retrieved from output queue
INFO - 17:45:51: accumulated word occurrence stats for 232258 virtual documents



Coherence Score using c_v:  0.3263054753291283

Coherence Score using u_mass:  -1.4572390267580078


In [11]:
a=pyLDAvis.gensim.prepare(lda_model, corpus, words)
pyLDAvis.show(a)

AttributeError: module 'pyLDAvis' has no attribute 'gensim'